In [1]:
import numpy as np
import pandas as pd
import string
import spacy
import pickle

import dask.dataframe as dd
from dask.distributed import Client
from dask_ml.model_selection import train_test_split
from dask_ml.feature_extraction.text import HashingVectorizer
from dask_ml.wrappers import Incremental
from dask_ml.metrics import accuracy_score

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
from sklearn.linear_model import SGDClassifier,Perceptron
from sklearn.metrics import classification_report


In [2]:
from dask.distributed import Client

client = Client(threads_per_worker=2,
                n_workers=5, memory_limit='3GB')
client

Client Scheduler: tcp://127.0.0.1:39715 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 10 Memory: 15.00 GB


In [3]:
# Importation des `noms de métiers`
# Création du dico des `noms de métiers`
jobNames = pd.read_csv('./data/categories_string.csv')['0'].to_dict()


# Importation du jeu de données d'entrainement

trainX=(dd.from_pandas(pd.read_json('./data/train.json',dtype={'description':'string','gender':'string'})
          .set_index('Id'), npartitions=11))

trainY=dd.read_csv('./data/train_label.csv').set_index('Id',npartitions=11)

# Concatenation en un seul DataFrame pour visualiser
trainDF= trainX.assign(Y = trainY.Category).categorize(columns=['gender'])

In [6]:
X = trainDF['description']
ylabels = trainDF['Y'] 

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2,shuffle=True)

In [11]:
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

In [12]:
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [13]:
from dask_ml.feature_extraction.text import HashingVectorizer
hw_vector = HashingVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1, 2), n_features=2**20)

In [14]:
%time
Feature_pipeline = Pipeline([('vectorizer', hw_vector)])
Pipeline_Model = Feature_pipeline.fit(X_train.values)

CPU times: user 19 µs, sys: 3 µs, total: 22 µs
Wall time: 41 µs


/home/pierre/anaconda3/envs/ProjetSD/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [15]:
Text_preprocess_pipe = pickle.dumps(Pipeline_Model)

In [16]:
Pipeline_Model = pickle.loads(Text_preprocess_pipe)

In [17]:
%time
X_transformed = Pipeline_Model.transform(X_train)

CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 21.5 µs


{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27}

In [ ]:
%%time
import joblib
estimator = Perceptron(n_jobs=-1)
classifier = Incremental(estimator)
Model = classifier.fit(X_transformed,
               y_train,classes=y_train)

/home/pierre/anaconda3/envs/ProjetSD/lib/python3.7/site-packages/distributed/worker.py:3382: UserWarning: Large object of size 91.43 MB detected in task graph: 
  ("('fit-2d3182995b6804b9f35395d3df160b5f', 9)", "( ... it, 114 tasks})
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)
